Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
df[:condition] = convert(Array{Int64}, df[:condition])
df[:actor] = convert(Array{Int64}, df[:actor])
first(df, 5)

struct m_10_04d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TA <: AbstractVector}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Actors"
    A::TA
    "Number of observations"
    N::Int
    "Number of unique actors"
    N_actors::Int
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_04d_model)(θ)
    @unpack y, X, A, N, N_actors = problem   # extract the data
    @unpack β, α = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # bp & bpC
    ll += sum(logpdf.(Normal(0, 10), α)) # alpha[1:7]
    ll += sum(
      [loglikelihood(Binomial(1, logistic(α[A[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_actors = length(unique(df[:actor]))
X = hcat(ones(Int64, N), df[:prosoc_left] .* df[:condition]);
A = df[:actor]
y = df[:pulled_left]
p = m_10_04d_model(y, X, A, N, N_actors);
θ = (β = [1.0, 0.0], α = [-1.0, 10.0, -1.0, -1.0, -1.0, 0.0, 2.0])
p(θ)

-305.21943396408915

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_04d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_actors), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = LogDensityRejectErrors(ADgradient(:Flux, P));
#∇P = ADgradient(:ForwardDiff, P);
#∇P = ADgradient(:Flux, P);

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0099 s/step ...done
MCMC, adapting ϵ (25 steps)
0.015 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0082 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0036 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0026 s/step ...done
MCMC, adapting ϵ (400 steps)
0.0023 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0032 s/step ...done
MCMC (1000 steps)
step 393 (of 1000), 0.0025 s/step
step 778 (of 1000), 0.0026 s/step
0.0026 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [-4.32011, 0.230806], α = [3.57893, 7.82109, 3.63461, 3.76537, 4.08817, 5.11468, 6.03383])      
 (β = [-1.77483, 0.393542], α = [1.63729, 16.7868, 1.05232, 1.54289, 1.23874, 2.3266, 3.95026])       
 (β = [1.36297, 0.0668481], α = [-1.93286, 13.5634, -1.8614, -2.38328, -1.73653, -0.835717, 0.317655])
 (β = [4.59924, 0.705946], α = [-4.9042, -0.435958, -5.83329, -5.26582, -5.14374, -4.02197, -2.43733])
 (β = [4.24455, 0.329633], α = [-5.00585, 6.07366, -4.86582, -5.30469, -4.34947, -3.94576, -2.00868]) 

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 1.5128727628091099 
 0.42561201916954794

Extract the parameter posterior means: `β`,

In [10]:
posterior_α = mean(last, posterior)

7-element Array{Float64,1}:
 -1.9542073122803718
 10.692312146741594 
 -2.2442118199400207
 -2.265604541056308 
 -1.9548381381519657
 -1.0258427065959832
  0.527027017653806 

Effective sample sizes (of untransformed draws)

In [11]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×9 Array{Float64,2}:
 471.682  1000.0  467.256  395.161  …  468.123  468.291  471.971  462.807

NUTS-specific statistics

In [12]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.9, min/25%/median/75%/max: 0.0 0.87 0.95 0.99 1.0
  termination: AdjacentDivergent => 1% AdjacentTurn => 34% DoubledTurn => 66%
  depth: 2 => 2% 3 => 56% 4 => 40% 5 => 2% 6 => 0%


Result rethinking

In [13]:
rethinking = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Means of draws

In [14]:
[posterior_β, posterior_α]

2-element Array{Array{Float64,1},1}:
 [1.51287, 0.425612]                                                 
 [-1.95421, 10.6923, -2.24421, -2.2656, -1.95484, -1.02584, 0.527027]

End of `10/m10.04d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*